# Student Loan Risk with Deep Learning

In [7]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [9]:
# Read the CSV into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
print("DataFrame Overview:")
display(loans_df.head())

DataFrame Overview:


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [11]:
# Review the data types associated with the columns
print("Data Types of Columns:")
print(loans_df.dtypes)

Data Types of Columns:
payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object


In [13]:
# Check the credit_ranking value counts
print("Value Counts for 'credit_ranking':")
print(loans_df["credit_ranking"].value_counts())

Value Counts for 'credit_ranking':
credit_ranking
1    855
0    744
Name: count, dtype: int64


### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [15]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [17]:
# Define the features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=["credit_ranking"])

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [19]:
from sklearn.model_selection import train_test_split

# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Review the shape of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1279, 11)
X_test shape: (320, 11)
y_train shape: (1279,)
y_test shape: (320,)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [21]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)

# Scale the features testing dataset
X_test_scaled = scaler.transform(X_test)

# Verify the scaling by checking a sample of the scaled training data
pd.DataFrame(X_train_scaled, columns=X.columns).head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,-0.733079,0.664893,-1.257044,-0.320459,-0.453622,-0.742407,-0.645507,0.240001,0.988460,0.063095,-0.872234
1,1.067741,-0.623462,1.523148,0.608863,-0.369546,-1.125190,-1.112003,0.187899,-1.753513,-0.173904,-0.779785
2,-1.746040,-1.071585,-1.358142,-0.534917,-0.789922,1.075808,0.536281,-2.677737,2.327564,0.774090,3.287990
3,-0.620528,0.496847,-1.054849,-0.034513,-0.201396,0.118853,1.189375,0.375468,1.243528,-0.766400,-0.687335
4,-0.507977,0.608877,-1.004300,-0.534917,0.261018,-0.742407,-0.707707,-0.333126,-0.095575,-0.470152,-0.779785


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [23]:
# Define the number of inputs (features) to the model
number_input_features = X_train_scaled.shape[1]

# Review the number of features
print(f"Number of input features: {number_input_features}")

Number of input features: 11


In [25]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 16

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 8

# Define the number of neurons in the output layer
output_neurons = 1  # Binary classification (repayment success or failure)

In [29]:
from tensorflow.keras import Input

# Create the Sequential model instance
nn_model = Sequential()

# Add an Input layer to specify the shape of the inputs
nn_model.add(Input(shape=(number_input_features,)))

# Add the first hidden layer
nn_model.add(Dense(units=hidden_nodes_layer1, activation="relu"))

# Add the second hidden layer
nn_model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model
nn_model.add(Dense(units=output_neurons, activation="sigmoid"))

In [31]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 16)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 337 (1.32 KB)

 Trainable params: 337 (1.32 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [34]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [36]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, verbose=1)

Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step - accuracy: 0.5381 - loss: 0.7014 
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 922us/step - accuracy: 0.6158 - loss: 0.6629
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - accuracy: 0.6632 - loss: 0.6267
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 812us/step - accuracy: 0.6848 - loss: 0.6006
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - accuracy: 0.6993 - loss: 0.5722
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - accuracy: 0.7138 - loss: 0.5533
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step - accuracy: 0.7359 - loss: 0.5338
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step - accuracy: 0.7198 - loss: 0.5242
Epoch 9/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - accuracy: 0.7257 - loss: 0.5164
Epoch 10/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step - accuracy: 0.7111 - loss: 0.5359
Epoch 11/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step - accuracy: 0.7381 - loss: 0.5015
Epoch 12/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/ste

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [39]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - 8ms/step - accuracy: 0.7406 - loss: 0.5168
Loss: 0.5167719125747681, Accuracy: 0.7406250238418579


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [42]:
# Set the model's file path
file_path = "student_loans.keras"

# Export your model to a keras file
nn_model.save(file_path)
print(f"Model saved to: {file_path}")

Model saved to: student_loans.keras


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [44]:
# Set the model's file path
model_path = "student_loans.keras"

# Load the model to a new object
loaded_model = tf.keras.models.load_model(model_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [47]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Display a sample of the predictions
print(predictions[:10])  # Display the first 10 predictions

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
[[0.37493306]
 [0.46821338]
 [0.92807466]
 [0.78221726]
 [0.9888912 ]
 [0.8497619 ]
 [0.93090004]
 [0.09102902]
 [0.47415808]
 [0.21619979]]


In [49]:
# Save the predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=["Prediction"])

# Round the predictions to binary results (0 or 1)
predictions_df["Binary_Prediction"] = (predictions_df["Prediction"] > 0.5).astype(int)

# Display a sample of the predictions DataFrame
print(predictions_df.head())

   Prediction  Binary_Prediction
0    0.374933                  0
1    0.468213                  0
2    0.928075                  1
3    0.782217                  1
4    0.988891                  1


### Step 4: Display a classification report with the y test data and predictions

In [52]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["Binary_Prediction"]))

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       154
           1       0.77      0.71      0.74       166

    accuracy                           0.74       320
   macro avg       0.74      0.74      0.74       320
weighted avg       0.74      0.74      0.74       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

In [63]:
'''
1. To build a recommendation system for student loan options, the following data would be required:

Demographic Data: Age, location, marital status, dependents, and employment status. This helps understand the student's financial responsibilities and eligibility for different loan options.
Educational Data: Current l
evel of education, program of study, institution type, and expected graduation date. This is critical for tailoring loans to fit the student’s educational journey.
Financial Data: Annual income, credit score, existing debt, and savings. This data ensures the recommendations are affordable and realistic based on the student's financial situation.
Loan Preferences: Desired loan amount, preferred repayment period, and interest rate preferences. This ensures the recommendations align with the student’s goals.
Behavioral Data: Previous loan history, repayment patterns, and spending habits. This helps in predicting repayment likelihood and designing personalized recommendations.
This data is relevant because it provides a holistic view of the student’s needs, financial situation, and repayment capacity, allowing for accurate and personalized loan recommendations.
'''

"\n1. To build a recommendation system for student loan options, the following data would be required:\n\nDemographic Data: Age, location, marital status, dependents, and employment status. This helps understand the student's financial responsibilities and eligibility for different loan options.\nEducational Data: Current l\nevel of education, program of study, institution type, and expected graduation date. This is critical for tailoring loans to fit the student’s educational journey.\nFinancial Data: Annual income, credit score, existing debt, and savings. This data ensures the recommendations are affordable and realistic based on the student's financial situation.\nLoan Preferences: Desired loan amount, preferred repayment period, and interest rate preferences. This ensures the recommendations align with the student’s goals.\nBehavioral Data: Previous loan history, repayment patterns, and spending habits. This helps in predicting repayment likelihood and designing personalized recom

In [ ]:
'''
2. The model would use context-based filtering.

Justification: Context-based filtering tailors recommendations by considering multiple contextual features (e.g., demographics, financial data, and loan preferences).
In this case, loan recommendations depend on a combination of financial capability, educational goals, and behavioral data.
Collaborative filtering (based on shared patterns among users) or content-based filtering (focused only on loan features) wouldn't fully capture the personalized context needed for this system.
'''

In [ ]:
'''
3A. Data Privacy and Security:
Concern: Collecting sensitive financial and personal data (e.g., income, credit scores, educational history) raises significant privacy concerns.
Ensuring secure storage, compliance with regulations (like GDPR or CCPA), and gaining user trust is essential to prevent data breaches.

Why it’s a challenge:
Handling sensitive data improperly could lead to legal consequences and loss of user trust.

3B. Bias and Fairness:
Concern: The system might unintentionally favor certain demographics or institutions,
    leading to unfair recommendations for underserved groups (e.g., low-income students, students from smaller institutions).
Why it’s a challenge:
Bias in recommendations could harm students by excluding them from fair access to loan opportunities and could damage the reputation of the recommendation system.
'''